# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [26]:
# 하기 코드실행에 필요한 라이브러리들을 가져오는 코드입니다.

import cv2, imutils, re, sys, math
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams
import operator
import heapq
from pathlib import Path

In [27]:
# 현재의 디렉토리를 확인합니다. 디렉토리를 확인하고 이미지를 가져올 폴더를 선택합니다.
img_dir = '..\data\processed_images\Part2'

### Using json to parse the captions and dois for each image

In [28]:
# url 데이터를 담을 json 파일을 설정합니다. (선택)

#with open('C:/Users/user/Desktop/TIPA/code-20220207T014959Z-001/code/data/urldata.json') as handler:
#    urldata = json.load(handler)

In [29]:
# doi 데이터를 담을 json 파일을 설정합니다. (필수)

with open(r'..\data\doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [30]:
# ocr image text 데이터를 담을 json 파일을 설정합니다. (필수)

with open(r'..\data\ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

In [31]:
# aws rekognition text 데이터를 담을 json 파일을 설정합니다. (필수)

with open(r'..\data\aws-rekognition-output.json') as awshandler:
    bbox_text = json.load(awshandler)

### Function to detect x-axis and y-axis for a plot

Axes detection

In [32]:
def axisreader(filepath, threshold = None, number_of_elements = None):

    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
        
    if number_of_elements is None:
        number_of_elements = 100
    
    img = cv2.imread(filepath)

    height, width, channels = img.shape
    crop_img = img[3:height-3, 3:width-3]

# 각각 RGB 값의 value 가 +- 1 인 threshold 만들기.

    higher_threshold = np.median(crop_img, axis=2) + threshold
    lower_threshold = np.median(crop_img, axis=2) - threshold

# RGB 중에 한 값이라도 250을 넘어가면 전부 255,255,255로 처리하고, 0보다 작거나 같은 값은 0,0,0으로 처리해라.
    np.place(higher_threshold, higher_threshold > 250, [255,255,255])
    np.place(lower_threshold, lower_threshold <= 0, [0,0,0])

    higher_threshold = np.rint(higher_threshold).astype(int)
    lower_threshold = np.rint(lower_threshold).astype(int)

    h1 = np.array([higher_threshold, higher_threshold, higher_threshold]).astype(int)
    l1 = np.array([lower_threshold,lower_threshold,lower_threshold]).astype(int)

    h1 = h1.reshape(height-6,width-6,3)
    l1 = l1.reshape(height-6,width-6,3)

# RGB 값이 같거나, threshold 안쪽 범위이거나, white 가 아니라면 1로, 그 외에는 0 으로 처리.

    mask = np.uint8(np.where(((crop_img != 255) & (crop_img == crop_img)) | ((crop_img < h1) & (l1 <= crop_img)) , 1, 0))

# reduce sum으로 각각의 row, column 를 더한 후, 조건을 만족하는 1 값이 가장 많은 row, column 찾기

    col_counts = cv2.reduce(mask, 0, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)
    row_counts = cv2.reduce(mask, 1, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)

    col_list = col_counts.flatten().tolist()
    row_list = row_counts.flatten().tolist()

    max_col = max(col_list)
    max_row = max(row_list)

    # c = col_counts.flatten().tolist()
#############################################################################################
    # col_max_index_list = [index for index in range(len(col_list)) if col_list[index] == max_col]

    col_set = set(col_list)
    col_list_fromset = list(col_set)

    col_list_max = heapq.nlargest(number_of_elements, col_list_fromset)


    col_list_maxnum = []
    for i in col_list_max:
        if i < max(col_list_max)-50:
            continue
        col_list_maxnum.append(i)

    col_list_maxnum_norepeat = []
    for v in col_list_maxnum:
        if v not in col_list_maxnum_norepeat:
            col_list_maxnum_norepeat.append(v)


    col_max_index_listnum = []
    for i in col_list:
        for a in col_list_maxnum_norepeat:
            if i == a:
                x = [index for index in range(
                    len(col_list)) if col_list[index] == i]
                for i in x:
                    col_max_index_listnum.append(i)

    col_max_index_listnum

#############################################################################################
    c1 = round(col_max_index_listnum[0]/3)
    yaxis = (c1, 0, c1, height-6)

    # r = row_counts.flatten().tolist()
#############################################################################################
    # row_max_index_list = [index for index in range(len(row_list)) if row_list[index] == max_row]

    row_set = set(row_list)
    row_list_fromset = list(row_set)

    row_list_max = heapq.nlargest(number_of_elements, row_list_fromset)

    row_list_maxnum = []
    for i in row_list_max:
        if i < max(row_list_max)-50:
            continue
        row_list_maxnum.append(i)

    row_list_maxnum_norepeat = []
    for v in row_list_maxnum:
        if v not in row_list_maxnum_norepeat:
            row_list_maxnum_norepeat.append(v)


    row_max_index_listnum = []
    for i in row_list:
        for a in row_list_maxnum_norepeat:
            if i == a:
                x = [index for index in range(len(row_list)) if row_list[index] == i]
                for i in x:
                    row_max_index_listnum.append(i)

    row_max_index_listnum
##############################################################################################
    r1 = round(row_max_index_listnum[-1]/3)
    xaxis = (0, r1, width-6, r1)


# 선그리기

    #cv2.line(crop_img, (0,round(r1)), (w, round(r1)), color=(0,0,255), thickness=1)
    
    #cv2.line(crop_img, (round(c1),0), (round(c1),h), color=(0,0,255), thickness=1)
    
    
    return xaxis, yaxis

In [33]:
def cleanText(image_text):
    
    return [(text, (textx, texty, w, h)) for text, (textx, texty, w, h) in image_text if text.strip() != 'I']

In [34]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    image_text = cleanText(image_text)
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'^[+-]?\d+(?:\.\d+)?[%-]?$', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    x_labels_list = maxList.copy()
    
    x_text = x_labels.copy()
    x_labels = []
    hmax = 0
    
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        if texty + h > hmax:
            hmax = texty + h
    
    # Get possible x-text by moving from where we
    # left off in x-labels to the complete
    # height of the image.
    maxIntersection = 0
    maxList = []
    for i in range(hmax + 1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_text):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = []
    for text, (textx, texty, w, h) in maxList:
        x_text.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    legends_and_numbers = mergeTextBoxes(legends)
    
    legends = []
    for text, (textx, texty, w, h) in legends_and_numbers:
        if not re.search(r'^([(+-]*?(\d+)?(?:\.\d+)*?[-%) ]*?)*$', text):
            legends.append((text, (textx, texty, w, h)))
    
    
    def canMerge(group, candidate):
        candText, candRect = candidate
        candx, candy, candw, candh = candRect
        
        for memText, memRect in group:
            memx, memy, memw, memh = memRect
                
            if abs(candy - memy) <= 5 and abs(candy + candh - memy - memh) <= 5:
                return True
            elif abs(candx - memx) <= 5:
                return True
                
        return False
    
    # Grouping Algorithm
    legend_groups = []
    for index, (text, rect) in enumerate(legends):
        #print("text: {0}, rect: {1}\n".format(text, rect))
        
        for groupid, group in enumerate(legend_groups):
            if canMerge(group, (text, rect)):
                group.append((text, rect))
                break
        else:
            legend_groups.append([(text, rect)])
    
    #print(legend_groups)
    #print("\n\n")
    
    maxList = []
    
    if len(legend_groups) > 0:
        maxList = max(legend_groups, key = len)

    legends = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
        
    return image, x_labels, x_labels_list, x_text, y_labels, y_labels_list, y_text_list, legends, maxList

In [35]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* Finally, the value-tick ratio to normalize the heights of the bounding boxes is calculated by:  

$$
ratio  = \frac{\Delta ticks_{average} }{y\;ticks_{average}}
$$


In [36]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [37]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [38]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook 엑셀로 저장하기

In [39]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 비율 맞추기

Y-val data:
    - The height of each bounding box is recorded by the help of the merging rectangles during Cluster count 
    estimation method.
    - Eventually, we used the ratio to calculate the y-values as:

$$
y_{values}  =  ratio \times height_{bounding\;box}
$$


In [40]:
def mergeRects(contours, mode='contours'):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        if mode == 'contours':
            rects.append(cv2.boundingRect(cnt))
        elif mode == 'rects':
            rects.append(cnt)
        
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])
    
    return acceptedRects

In [41]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])
    
    return acceptedRects

In [42]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [43]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    return abs(rectAx + rectAx / 2 - rectBx - rectBx / 2)

In [44]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

In [45]:
def expand(points, margin):
    return np.array([
        [[points[0][0][0] - margin, points[0][0][1] - margin]],
        [[points[1][0][0] + margin, points[1][0][1] - margin]],
        [[points[2][0][0] + margin, points[2][0][1] + margin]],
        [[points[3][0][0] - margin, points[3][0][1] + margin]]])

## Saving y-values in our data excel sheet 엑셀로 저장하기

In [46]:
images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg') or path.name.endswith('.PNG'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_height, img_width, _ = img.shape
            
            # Axes detection
            xaxis, yaxis = axisreader(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, x_labels_list, _, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            actual_image = img.copy()
            
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')
            
                texts = bbox_text[path.name]['TextDetections']
                
                for text in texts:
                    if text['Type'] == 'WORD' and text['Confidence'] >= 80:
                        vertices = [[vertex['X'] * img_width, vertex['Y'] * img_height] for vertex in text['Geometry']['Polygon']]
                        vertices = np.array(vertices, np.int32)
                        vertices = vertices.reshape((-1, 1, 2))

                        img = cv2.fillPoly(img, [expand(vertices, 1)], (255, 255, 255))

                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                threshold = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
                
                contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = [contour for contour in contours if cv2.contourArea(contour) < 0.01 * img_height * img_width]

                contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]
                rects = [cv2.boundingRect(contour) for contour in contours]

                groups = []
                legendtexts = []
                legendrects = []
    
                for box in legendBoxes:
                    text, (textx, texty, width, height) = box
                    bboxes = filterBbox(rects, box)
                    
                    if bboxes is not None:
                        for rect in [bboxes]:
                            (x, y, w, h) = rect
                            legendrects.append(rect)
                            
                            group = boxGroup(actual_image, rect)[0]
                            group = [arr.tolist() for arr in group]
                            
                            groups.append(group)
                            legendtexts.append(text)
                            
                            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)

                        cv2.rectangle(img, (textx, texty), (textx + width, texty + height), (255, 0, 0), 2)
                                 
                data = {}
                for legend in legends:
                    data[legend] = {}
                    
                    for x_label, box in x_labels_list:
                        data[legend][x_label] = 0.0
                        
                for i in range(len(groups)):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    legendtext = legendtexts[i]
                    
                    for box in legendrects:
                        (textx, texty, width, height) = box
                        cv2.rectangle(img, (textx, texty), (textx + width, texty + height), (255, 255, 255), cv2.FILLED)
                    
                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    labels = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        labeltext = None

                        for text, textBox in x_labels_list:
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                labeltext = text

                        textBoxes.append(closestBox)
                        labels.append(labeltext)
                        
                    list_len = []
                    
                    for rect in rects:
                        list_len.append((rect, float(rect[3])))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [(rect, round(l* normalize_ratio, 1)) for rect, l in list_len]
                    
                    for x_label, box in x_labels_list:
                        (x, y, w, h) = box
                        value = 0.0
                        closest = None
                        dist = sys.maxsize
                        
                        for index, item in enumerate(y_val):
                            if labels[index] == x_label:
                                (vx, vy, vw, vh) = item[0]
                                if abs(x + w/2 - vx - vw/2) < dist:
                                    dist = abs(x + w/2 - vx - vw/2)
                                    closest = item[0]
                                    value = item[1]
                             
                        data[legendtext][x_label] = value
                     
                yValueDict[path.name] = data
            
            except Exception as e:
                print(e)
                continue
                
    return yValueDict

In [47]:
def filterBbox(rects, legendBox):
    text, (textx, texty, width, height) = legendBox
    
    filtered = []
    for rect in rects:
        (x, y, w, h) = rect
        if abs(y - texty) <= 10 and abs(y - texty + h - height) <= 10:
            filtered.append(rect)

    filtered = mergeRects(filtered, 'rects')
    
    closest = None
    dist = sys.maxsize
    for rect in filtered:
        (x, y, w, h) = rect
        if abs(x + w - textx) <= dist:
            dist = abs(x + w - textx)
            closest = rect
    
    return closest

In [48]:
def boxGroup(img, box):
    (x, y, w, h) = box

    image = img[y:y+h, x:x+w].reshape((h * w, 3))
    values, counts = np.unique(image, axis = 0, return_counts = True)

    # Remove white and near-by pixels
    threshold = 5
    for r in range(255 - threshold, 256):
        for g in range(255 - threshold, 256):
            for b in range(255 - threshold, 256):
                image = image[np.where((image != [r, g, b]).any(axis = 1))]

    values, counts = np.unique(image, axis = 0, return_counts = True)
                
    sort_indices = np.argsort(-counts)
    values, counts = values[sort_indices], counts[sort_indices]

    groups = []
    groupcounts = []

    for idx, value in enumerate(values):
        grouped = False

        for groupid, group in enumerate(groups):
            for member in group:
                r, g, b = member
                vr, vg, vb = value

                if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                    abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                    abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                        group.append(value)
                        groupcounts[groupid] += counts[idx]
                        grouped = True
                        break

            if grouped:
                break

        if not grouped:
            groups.append([value])
            groupcounts.append(counts[idx])

    groupcounts = np.array(groupcounts)
    sort_indices = np.argsort(-groupcounts)
    new_groups = [groups[i] for i in sort_indices]
    groups = new_groups
    
    return groups

In [49]:
# 뽑아낸 정보 dictionary 로 저장하기

yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [56.0, 58.0, 56.0, 58.0, 56.0], text_diff: [25.0, 25.0, 25.0, 25.0, 25.0]
[reject_outliers] ticks_diff: [56. 56. 56.], text_diff: [25. 25. 25. 25. 25.]
[getYVal] legends: ['Irbesartan']
[0] path: fig-1-2_PNG.rf.eecf38f6dcea5d592a8285895d979c84.jpg, ratio: 0.44642857142857145

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['1,000']
[1] path: fig-1-A_PNG.rf.5867c808e21ec09a6fa4587594b98962.jpg, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['*']
[2] path: fig-1-B_PNG.rf.1cb2c63027371ddd3d187f75310ba6aa.jpg, ratio: nan

[get text-to-tick ratio] ticks_diff: [49.0, 48.0, 49.0, 49.0, 49.0, 48.0, 49.0], text_diff: [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [49. 49. 49. 49. 49.], text_diff: [20. 20. 20. 20. 20. 20. 20.]
[getYVal] legends: []
[3] path: fig-2-B_PNG.rf.

C:\Users\user\AppData\Local\Temp/ipykernel_24812/3744830701.py:62: RuntimeWarning: Mean of empty slice.
  normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()


[getYVal] legends: ['*', '*']
[5] path: fig-4-A_PNG.rf.4ee7d6c6b93863329e10ee3bb77043ab.jpg, ratio: 0.11494252873563218

[get text-to-tick ratio] ticks_diff: [69.0, 68.0, 67.0, 67.0, 67.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [68. 67. 67. 67.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: []
[6] path: fig-5-B_PNG.rf.17258dc9ca3161290f54a18ba0cd936b.jpg, ratio: 0.14869888475836432

[get text-to-tick ratio] ticks_diff: [48.0, 48.0, 47.0, 48.0, 47.0, 48.0], text_diff: [50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [48. 48. 48. 48.], text_diff: [50. 50. 50. 50. 50. 50.]
[getYVal] legends: ['SHR', 'SHR + HDZ']
[7] path: fig-6-b_PNG.rf.ec1abe40f7961704ff8d203b9cdd2cc8.jpg, ratio: 1.0416666666666667

[get text-to-tick ratio] ticks_diff: [21.0, 23.0, 22.0, 22.0, 23.0, 22.0, 22.0, 22.0, 22.0, 23.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [22. 22. 22. 22. 22. 22.], text_d

In [50]:
workbook = xlsxwriter.Workbook(r'../results/FigureData.xlsx', {'nan_inf_to_errors': True})

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg') or path.name.endswith('.PNG'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = axisreader(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, _, x_text, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            pdfname = '-'.join(path.name.split('-')[:-2]) + '.pdf'
            
            if pdfname in doidata['Doi']:
                doi = doidata['Doi'][pdfname]
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi", doi, 0)
            addToExcel("file name", [path.name], 1)
            addToExcel("x-text", x_text, 2)
            addToExcel("x-labels", x_labels, 3)
            addToExcel("y-text", y_text, 4)
            addToExcel("y-labels", y_labels, 5)
            addToExcel("legends", legends, 6)
            
            data = yValueDict[path.name]

            column = 9
            for legend, datadict in data.items():
                if column == 9:
                    addToExcel("", datadict.keys(), 8)    
                    
                addToExcel(legend, datadict.values(), column)
                column += 1
            
            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-text       :  ", x_text)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("data         :  ", data, end= "\n\n")
            
            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: fig-1-2_PNG.rf.eecf38f6dcea5d592a8285895d979c84.jpg
file name    :   fig-1-2_PNG.rf.eecf38f6dcea5d592a8285895d979c84.jpg
doi          :   
x-text       :   ['Log', '(mol/L)']
x-labels     :   ['-6', '-5', 'do']
y-text       :   ['Response', '(%)']
y-labels     :   ['125', '100', '75', '50', '25', '0']
legends      :   ['Irbesartan']
data         :   {'Irbesartan': {'-6': 0.0, '-5': 0.0, 'do': 0.0}}

[1] path: fig-1-A_PNG.rf.5867c808e21ec09a6fa4587594b98962.jpg
file name    :   fig-1-A_PNG.rf.5867c808e21ec09a6fa4587594b98962.jpg
doi          :   
x-text       :   ['Time', '(h)']
x-labels     :   ['0', '12', '24', '36', '48']
y-text       :   ['Fimasartan', 'concentration']
y-labels     :   []
legends      :   ['1,000']
data         :   {'1,000': {'0': 0.0, '12': 0.0, '24': 0.0, '36': 0.0, '48': 0.0}}

[2] path: fig-1-B_PNG.rf.1cb2c63027371ddd3d187f75310ba6aa.jpg
file name    :   fig-1-B_PNG.rf.1cb2c63027371ddd3d187f75310ba6aa.jpg
doi          :   
x-text       :   ['ARB', 'con

새로고침